In [5]:
import paramiko

# Replace with your details
TAILSCALE_IP = "100.104.54.62"   # Tailscale IP of target
USERNAME = "juanincho"   # e.g. pi, ubuntu, etc.
PASSWORD = "2314"   # or use a key

# Initialize SSH client
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    print(f"Connecting to {TAILSCALE_IP} via SSH...")
    client.connect(TAILSCALE_IP, username=USERNAME, password=PASSWORD)

    stdin, stdout, stderr = client.exec_command("hostname && uptime")
    print("Output:")
    print(stdout.read().decode())

except Exception as e:
    print("Connection failed:", e)
finally:
    client.close()


Connecting to 100.104.54.62 via SSH...
Output:
ubuntu
 05:43:01 up  2:13,  2 users,  load average: 0.95, 0.64, 0.47



## Prueba mandando el array de las 4 esquinas

In [3]:
import paramiko
import json
import sys
import time

In [4]:
# Replace with your details
TAILSCALE_IP = "100.104.54.62"   # Tailscale IP of target
USERNAME = "juanincho"   # e.g. pi, ubuntu, etc.
PASSWORD = "2314"   # or use a key

# Initialize SSH client
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

#poniendo array con las esquinas en un json
json_payload = json.dumps([(20.432822, -99.599167), (20.432822, -99.599031), (20.432735, -99.599167), (20.432735, -99.599031)])

try:
    # Ejecutar el script remoto y enviarle JSON por stdin
    # 'python3 -u' para modo sin buffer y que la salida venga inmediatamente
    cmd = f"bash -lc 'source /home/juanincho/Documents/GitHub/proyecto_drone/venv/bin/activate && python3 -u /home/juanincho/Documents/GitHub/proyecto_drone/test_receiving_path.py'"
    client.connect(TAILSCALE_IP, username=USERNAME, password=PASSWORD)
    stdin, stdout, stderr = client.exec_command(cmd, get_pty=False)

    # enviar JSON y cerrar stdin para indicar EOF
    stdin.write(json_payload)
    stdin.flush()
    stdin.channel.shutdown_write()

    # leer salida (bloqueante hasta que el proceso termine)
    out = stdout.read().decode('utf-8')
    err = stderr.read().decode('utf-8')

    if err:
        # opcionalmente mostrar errores del lado remoto
        print("REMOTE STDERR:", err, file=sys.stderr)

    try:
        response = json.loads(out.strip() or "{}")
    except json.JSONDecodeError:
        response = {"raw_stdout": out, "stderr": err}

finally:
    client.close()

print("Respuesta remota:", response)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

REMOTE STDERR: CRITICAL:autopilot:APM:Copter V3.3 (d6053245)
CRITICAL:autopilot:Frame: QUAD
ERROR:autopilot:ARMING MOTORS
CRITICAL:autopilot:Initialising APM...



In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.instance().kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [2]:
# Aumentar límite a, por ejemplo, 100 MB/s
%config ServerApp.iopub_data_rate_limit=100000000
